In [31]:
import pandas as pd
import geopandas as gpd
import folium
from json import loads, dumps
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from shapely.geometry import Point
import time
import googlemaps
import osmnx as ox
from sklearn.cluster import KMeans
import numpy as np

In [32]:
df = pd.read_csv('ibsj_sector.csv')
df = df.drop(['prefix', 'name', 'city', 'state'], axis=1)
#df = df[:39]

null_address = len(df.loc[df['address_1'].isnull()])
print("El archivo tiene {} registros".format(len(df)))
#print("Hay {} direcciones vacias en el archivo".format(null_address))

df = df[df['address_1'].notna()]
df = df[df['address_1'] != 'Actualizar']
df['address_1'] = df['address_1'].replace({'#':' ', 'C/':'Calle '}, regex=True)
df = df.reset_index()
print("Luego de eliminar los registros sin direccion, quedan {} registros".format(len(df)))



El archivo tiene 772 registros
Luego de eliminar los registros sin direccion, quedan 657 registros


In [ ]:

# Configurar la API de Google Maps
API_KEY = ""  # Reemplaza con tu clave de Google Maps
gmaps = googlemaps.Client(key=API_KEY)

# Función para obtener coordenadas y datos extra
def obtener_datos_direccion(direccion):
    try:
        geocode_result = gmaps.geocode(direccion, components={"country": "DO"})  # Restringido a República Dominicana
        if geocode_result:
            location = geocode_result[0]
            lat = location["geometry"]["location"]["lat"]
            lng = location["geometry"]["location"]["lng"]
            address_components = {comp["types"][0]: comp["long_name"] for comp in location["address_components"]}

            return pd.Series({
                "geometry": Point(lng, lat),
                "calle": address_components.get("route", ""),
                "numero": address_components.get("street_number", ""),
                "barrio": address_components.get("sublocality", address_components.get("political", address_components.get("sublocality_level_1", ""))),
                "ciudad": address_components.get("locality", address_components.get("administrative_area_level_2", "")),
                "estado": address_components.get("administrative_area_level_1", ""),
                "pais": address_components.get("country", ""),
                "codigo_postal": address_components.get("postal_code", "")
            })
    except Exception as e:
        print(f"Error con {direccion}: {e}")

    # Si falla, devolver valores vacíos
    return pd.Series({"geometry": None, "calle": "", "numero": "", "barrio": "", "ciudad": "", "estado": "", "pais": "", "codigo_postal": ""})


# Aplicar la función a cada dirección
df_info = df["address_1"].apply(obtener_datos_direccion)

# Unir los datos con el dataframe original
df = pd.concat([df, df_info], axis=1)

# Convertir a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")

# Eliminar filas sin coordenadas
#gdf = gdf[gdf["geometry"].notna()]

# Guardar resultado en un archivo CSV
gdf.to_csv("coordenadas_resultado.csv", index=False, encoding="utf-8")


In [34]:
coordinates = np.array([(point.y, point.x) for point in gdf.geometry])

In [35]:

def get_cluster_color(cluster_label):
    colors = [
        'purple', 'black', 'cadetblue', 'pink', 'red', 'blue', 'darkgreen', 
        'darkred', 'lightgreen', 'orange', 'beige', 'darkpurple', 'darkblue', 'green', 'gray', 'lightgray', 'lightred', 'lightblue'
    ]
    return colors[cluster_label]

# Cargar el archivo CSV con coordenadas y datos extra
df = gdf

# Asegurar que las coordenadas estén en formato correcto
# df["geometry"] = gpd.points_from_xy(df["geometry"].apply(lambda x: x.x), df["geometry"].apply(lambda x: x.y))
# gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")

# Crear un mapa centrado en República Dominicana
mapa = folium.Map(location=[18.7357, -70.1627], zoom_start=9)

# Extraer las coordenadas (latitud y longitud)
coordinates = np.array([(point.y, point.x) for point in gdf.geometry])

# Aplicar KMeans para clasificar los puntos en clusters
kmeans = KMeans(n_clusters=8)  # Cambia el número de clusters (5 en este caso)
gdf['cluster'] = kmeans.fit_predict(coordinates)


gdf_barrios = ox.features_from_place(
    "Ozama,  Dominican Republic",
    {"boundary": "administrative","admin_level": "6"}
)


# gdf_barrios = gdf_barrios[gdf_barrios['admin_level'] == '6']
gdf_barrios = gdf_barrios[gdf_barrios['name'].isin(
            ['Pedro Brand','Santo Domingo Norte', 
             'Distrito Nacional', 'Boca Chica',           
             'Los Alcarrizos','San Antonio de Guerra',
             'San Luis','Santo Domingo Este',
             'Santo Domingo Norte','Santo Domingo Oeste']
)]

# Agregar cada polígono al mapa
# for _, row in gdf_barrios.iterrows():
#     if row.geometry is not None:
#         folium.GeoJson(row.geometry, tooltip=row.get("name")).add_to(mapa)

gdf["poligono"] = gdf.geometry.apply(
    lambda punto: gdf_barrios[gdf_barrios.contains(punto)]["name"].values[0] 
    if not gdf_barrios[gdf_barrios.contains(punto)].empty else "Fuera de cualquier polígono"
)

# Agregar puntos al mapa con información detallada
for idx, row in gdf.iterrows():
    popup_text = f"""
    <b>Dirección Original:</b> {row["address_1"]}<br>
    <b>Nombre:</b> {row["first_name"]}<br>
    <b>Apellido:</b> {row["last_name"]}<br>
    <b>Calle:</b> {row["calle"]}<br>
    <b>Número:</b> {row["numero"]}<br>
    <b>Barrio:</b> {row["barrio"]}<br>
    <b>Ciudad:</b> {row["poligono"]}<br>
    <b>País:</b> {row["pais"]}<br>
    <b>Código Postal:</b> {row["codigo_postal"]}<br>
    <b>Cluster:</b> {row["cluster"]}
    """

    marker_color = get_cluster_color(row['cluster'])
    
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],  # Latitud, Longitud
        popup=folium.Popup(popup_text, max_width=300),  # Información al hacer clic
        tooltip=row["address_1"],  # Muestra la dirección al pasar el mouse
        icon=folium.Icon(color=marker_color, icon="info-sign")  # Color y estilo del marcador
    ).add_to(mapa)

# Guardar el mapa como HTML y mostrarlo
mapa.save("mapa_interactivo.html")

mapa


In [43]:
gdf.loc[gdf['poligono'] == 'Fuera de cualquier polígono'].to_csv('arreglar.csv',index=False)

In [42]:
gmaps = googlemaps.Client(key=API_KEY)
direccion = "Calle López Contreras 6, Herrera"  # Reemplázala con una dirección específica
geocode_result = gmaps.geocode(direccion, components={"country": "DO"})

location = geocode_result[0]
lat = location["geometry"]["location"]["lat"]
lng = location["geometry"]["location"]["lng"]
address_components = {comp["types"][0]: comp["long_name"] for comp in location["address_components"]}


df_test = pd.DataFrame(data={'lat': [lat], 'lng': [lng]})
df_test['geometry'] = Point(lng, lat)

mapa = folium.Map(location=[18.7357, -70.1627], zoom_start=9)
folium.Marker(
           location=[df_test.loc[0,'geometry'].y, df_test.loc[0,'geometry'].x]
        ).add_to(mapa)

mapa

In [38]:

import geopandas as gpd
import folium

# Descargar los polígonos de los barrios de Santo Domingo
gdf_barrios = ox.features_from_place(
    "Ozama,  Dominican Republic",
    {"boundary": "administrative","admin_level": "6"}
)
# gdf_barrios = ox.features_from_place(
#     "Santo Domingo, Dominican Republic",
#     {"boundary": "administrative", "admin_level": "10"}
# )

# gdf_barrios = gdf_barrios[gdf_barrios['admin_level'] == '6']
gdf_barrios = gdf_barrios[gdf_barrios['name'].isin(
            ['Pedro Brand','Santo Domingo Norte', 
             'Distrito Nacional', 'Boca Chica',           
             'Los Alcarrizos','San Antonio de Guerra',
             'San Luis','Santo Domingo Este',
             'Santo Domingo Norte','Santo Domingo Oeste']
)]

# gdf_barrios_filtered = gdf_barrios[gdf_barrios['name'].isin(
#     ['Distrito Nacional', 'Santo Domingo Este', 'Santo Domingo Oeste', 'Santo Domingo Norte', 'Los Alcarrizos'])]

# Crear un mapa centrado en Santo Domingo
mapa = folium.Map(location=[18.5, -69.9], zoom_start=9)

# Agregar cada polígono al mapa
for _, row in gdf_barrios.iterrows():
    if row.geometry is not None:
        folium.GeoJson(row.geometry, tooltip=row.get("name")).add_to(mapa)

# Mostrar el mapa
mapa.save("mapa_barrios_santo_domingo.html")
mapa
